In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [2]:
data = pd.read_csv('../data/WildfireData.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [3]:
data = data[['fire_size', 'remoteness', 'putout_time', 'stat_cause_descr']]
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
0,10.0,0.017923,NaN,Missing/Undefined
1,3.0,0.184355,NaN,Arson
2,60.0,0.194544,NaN,Arson
3,1.0,0.487447,0 days 00:00:00.000000000,Debris Burning
4,2.0,0.214633,NaN,Miscellaneous


In [4]:
data = data.dropna()
data['putout_time'] = data['putout_time'].str.split(' ').str[0]
data = data[data.stat_cause_descr != 'Miscellaneous']
data = data[data.stat_cause_descr != 'Missing/Undefined']
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
3,1.0,0.487447,0,Debris Burning
11,37.0,0.191923,0,Arson
24,40.0,0.199485,0,Arson
26,2.0,0.227270,0,Debris Burning
31,1.2,0.292303,0,Debris Burning


In [5]:
data.stat_cause_descr.unique()

array(['Debris Burning', 'Arson', 'Lightning', 'Equipment Use',
       'Campfire', 'Structure', 'Children', 'Smoking', 'Powerline',
       'Railroad', 'Fireworks'], dtype=object)

In [6]:
data['stat_cause_descr'] = data['stat_cause_descr'].map({
    'Debris Burning': 0,
    'Arson': 1,
    'Lightning': 2,
    'Equipment Use': 3,
    'Campfire': 4,
    'Structure': 5,
    'Children': 5,
    'Smoking': 5,
    'Powerline': 5,
    'Railroad': 5,
    'Fireworks': 5,
})
data.stat_cause_descr.unique()
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
3,1.0,0.487447,0,0
11,37.0,0.191923,0,1
24,40.0,0.199485,0,1
26,2.0,0.227270,0,0
31,1.2,0.292303,0,0


In [7]:
from sklearn.model_selection import train_test_split
X = data.drop('stat_cause_descr', axis=1).astype('float')
y = data.stat_cause_descr.astype('float')
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    fire_size  remoteness  putout_time
3         1.0    0.487447          0.0
11       37.0    0.191923          0.0
24       40.0    0.199485          0.0
26        2.0    0.227270          0.0
31        1.2    0.292303          0.0
3     0.0
11    1.0
24    1.0
26    0.0
31    0.0
Name: stat_cause_descr, dtype: float64


In [8]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
 y_test[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [10]:
model = Sequential()
model.add(Dense(32, input_dim=(3), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1)

Epoch 1/100
301/301 [==============================] - 15s 3ms/step - loss: 31.1978 - accuracy: 0.3064 - val_loss: 1.6605 - val_accuracy: 0.1838
Epoch 2/100
301/301 [==============================] - 0s 970us/step - loss: 1.7230 - accuracy: 0.2345 - val_loss: 1.5543 - val_accuracy: 0.4359
Epoch 3/100
301/301 [==============================] - 0s 1ms/step - loss: 1.5347 - accuracy: 0.4649 - val_loss: 1.5007 - val_accuracy: 0.4401
Epoch 4/100
301/301 [==============================] - 0s 1ms/step - loss: 1.4920 - accuracy: 0.4543 - val_loss: 1.4658 - val_accuracy: 0.4448
Epoch 5/100
301/301 [==============================] - 0s 1ms/step - loss: 1.4610 - accuracy: 0.4573 - val_loss: 1.4424 - val_accuracy: 0.4486
Epoch 6/100
301/301 [==============================] - 0s 1ms/step - loss: 1.4411 - accuracy: 0.4666 - val_loss: 1.4273 - val_accuracy: 0.4551
Epoch 7/100
301/301 [==============================] - 0s 1ms/step - loss: 1.4176 - accuracy: 0.4687 - val_loss: 1.4160 - val_accuracy: 0.

In [13]:
test_data = {
    'fire_size': [37.0],
    'remoteness': [0.191923],
    'putout_time': [0]	
}
test_data = pd.DataFrame(test_data)
test_data

,fire_size,remoteness,putout_time
0,37.0,0.191923,0


In [14]:
result = model.predict(test_data)
result

array([[0.4031258 , 0.46777004, 0.02570682, 0.05172341, 0.02056634,
        0.03110755]], dtype=float32)

In [15]:
final = 0
for i in range(6):
    if (result[0][i] > final):
        final = result[0][i]
    # print(final)
final = np.where(result[0] == final)
print(final[0][0])
    

1


In [16]:
model.save("../Weights/cause.h5")